In [13]:
import pandas as pd
import numpy as np
import re # Thư viện xử lý biểu thức chính quy (Regex) cho văn bản
import os


import sys
sys.path.append(os.path.abspath(r"..\src"))

import config as cf
import data_processing as dp 

In [26]:

# Tạo thư mục processed nếu chưa có
os.makedirs(os.path.dirname(cf.PROCESSED_DATA), exist_ok=True)

df = pd.read_csv(cf.RAW_DATA)
print(f"Dữ liệu gốc có kích thước: {df.shape}")
df.head(3)

Dữ liệu gốc có kích thước: (11033, 31)


,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,location_accuracy,landslide_category,...,country_code,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,submitted_date,created_date,last_edited_date,longitude,latitude
0,AGU,https://blogs.agu.org/landslideblog/2008/10/14...,684,08/01/2008 12:00:00 AM,NaN,"Sigou Village, Loufan County, Shanxi Province","occurred early in morning, 11 villagers buried...","Sigou Village, Loufan County, Shanxi Province",unknown,landslide,...,CN,Shaanxi,0.0,Jingyang,41.02145,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,107.4500,32.5625
1,Oregonian,http://www.oregonlive.com/news/index.ssf/2009/...,956,01/02/2009 02:00:00 AM,NaN,"Lake Oswego, Oregon",Hours of heavy rain are to blame for an overni...,"Lake Oswego, Oregon",5km,mudslide,...,US,Oregon,36619.0,Lake Oswego,0.60342,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-122.6630,45.4200
2,CBS News,https://www.cbsnews.com/news/dozens-missing-af...,973,01/19/2007 12:00:00 AM,NaN,"San Ramon district, 195 miles northeast of the...",(CBS/AP) At least 10 people died and as many a...,"San Ramon district, 195 miles northeast of the...",10km,landslide,...,PE,Junín,14708.0,San Ramón,0.85548,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-75.3587,-11.1295


In [27]:
# Danh sách các cột cần xóa
cols_to_drop = [
    'event_import_id', 
    'event_import_source', 
    'created_date', 
    'submitted_date', 
    'last_edited_date',
    'source_link', 
    'photo_link', 
    'storm_name',
    'country_code', 
    'event_time', 
    'notes',
    # 'gazeteer_closest_point', 
    # 'gazeteer_distance'
]

# Xóa cột nếu nó tồn tại trong dataframe
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

print(f"Kích thước sau khi xóa cột thừa: {df.shape}")

Kích thước sau khi xóa cột thừa: (11033, 20)


In [ ]:
# Chuyển đổi sang datetime, các giá trị lỗi sẽ biến thành NaT
df['event_date'] = pd.to_datetime(df['event_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

# Kiểm tra bao nhiêu dòng bị lỗi ngày tháng
missing_dates = df['event_date'].isna().sum()
print(f"Số lượng dòng bị lỗi ngày tháng: {missing_dates}")

print("Đã hoàn thành xử lý thời gian.")

Số lượng dòng bị lỗi ngày tháng: 0
Đã hoàn thành xử lý thời gian.


In [ ]:
# === Xử lý số liệu ===

# Với số người chết và bị thương, NaN thường nghĩa là không có thương vong được ghi nhận -> Điền 0
df['fatality_count'] = df['fatality_count'].fillna(0)
df['injury_count'] = df['injury_count'].fillna(0)

# Với dân số, điền bằng giá trị trung vị (median) để tránh bị ảnh hưởng bởi các thành phố quá lớn
# CÂN NHẮC TRƯỜNG NÀY
# XEM CÓ XỬ LÀM GÌ VỚI TRƯỜNG NÀY HAY KH 
# NẾU CÓ THÌ Điền median theo từng country:

# === Xử lý phân loại ===

# Với các cột phân loại, NaN nghĩa là không xác định -> Điền 'unknown'
cat_cols_to_fill = ['landslide_category', 'landslide_trigger', 'landslide_size', 'landslide_setting', 'country_name']
df[cat_cols_to_fill] = df[cat_cols_to_fill].fillna('unknown')

print("Đã xử lý xong Missing Values.")
print(df.isna().sum()) # Kiểm tra lại xem còn sót không

Đã xử lý xong Missing Values.
source_name                      0
event_id                         0
event_date                   11033
event_title                      0
event_description              862
location_description           102
location_accuracy                2
landslide_category               0
landslide_trigger                0
landslide_size                   0
landslide_setting                0
fatality_count                   0
injury_count                     0
country_name                     0
admin_division_name           1637
admin_division_population     1562
gazeteer_closest_point        1563
gazeteer_distance             1562
longitude                        0
latitude                         0
dtype: int64


In [35]:
df['landslide_size'].unique()


array(['large', 'small', 'medium', 'unknown', 'very_large', nan,
       'catastrophic'], dtype=object)

In [36]:
categorical_features = ['landslide_category', 'landslide_trigger', 'landslide_size', 'country_name', 'landslide_setting']

for col in categorical_features:
    df[col] = df[col].astype(str).str.lower().str.strip()

landslide_counts_raw = df['landslide_size'].value_counts()
# 2. Mapping/Gộp nhóm (Ví dụ: landslide_size)
# Đôi khi dữ liệu có các biến thể lạ, ta có thể map lại cho sạch

print("--- Số lượng vụ sạt lở theo mức độ (CHƯA GỘP) ---")
print(landslide_counts_raw)


--- Số lượng vụ sạt lở theo mức độ (CHƯA GỘP) ---
landslide_size
medium          6551
small           2767
unknown          851
large            750
very_large       102
nan                9
catastrophic       3
Name: count, dtype: int64


In [37]:
# 1. Chuẩn hóa các cột phân loại chính
# Chuyển về chữ thường (lowercase) và xóa khoảng trắng thừa (strip)
categorical_features = ['landslide_category', 'landslide_trigger', 'landslide_size', 'country_name', 'landslide_setting']


size_mapping = {
    'small': 'small',
    'medium': 'medium',
    'large': 'large',
    'very_large': 'very_large',
    'catastrophic': 'very_large', # Gộp catastrophic vào very_large
    'unknown': 'unknown',
    'nan': 'unknown'
}
# Chỉ map những giá trị có trong từ điển, còn lại giữ nguyên hoặc gán unknown
df['landslide_size'] = df['landslide_size'].map(size_mapping).fillna('unknown')
landslide_counts = df['landslide_size'].value_counts().sort_values(ascending=False)

print("Đã chuẩn hóa dữ liệu văn bản.")
print(df['landslide_size'].value_counts())

Đã chuẩn hóa dữ liệu văn bản.
landslide_size
medium        6551
small         2767
unknown        860
large          750
very_large     105
Name: count, dtype: int64


In [ ]:
# Áp dụng hàm làm sạch
# df['event_description'] = df['event_description'].apply(dp.clean_text)

# # Xem thử kết quả
# print(df['event_description'].head(2))

# Áp dụng hàm làm sạch nâng cao sau khi chạy hàm clean_text của bạn:
df['clean_description_nlp'] = df['clean_description'].apply(dp.advanced_nlp_cleaning)

print("Đã thêm cột 'clean_description_nlp' cho phân tích sâu hơn.")

                                   event_description  \
0  occurred early in morning, 11 villagers buried...   
1  Hours of heavy rain are to blame for an overni...   

                                   clean_description  
0  occurred early in morning 11 villagers buried ...  
1  hours of heavy rain are to blame for an overni...  


In [20]:
# Lọc chỉ lấy các dòng có tọa độ hợp lệ
df = df[
    (df['latitude'] >= -90) & (df['latitude'] <= 90) &
    (df['longitude'] >= -180) & (df['longitude'] <= 180)
]

print(f"Kích thước sau khi lọc tọa độ: {df.shape}")

Kích thước sau khi lọc tọa độ: (11033, 21)


In [21]:
# Lưu file
df.to_csv(cf.PROCESSED_DATA, index=False)

print(f"Đã lưu dữ liệu sạch tại: {cf.PROCESSED_DATA}")
print("Sẵn sàng cho bước 04!")

Đã lưu dữ liệu sạch tại: ..\data\processed\Global_Landslide_Processed.csv
Sẵn sàng cho bước 04!
